In [1]:
from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from backend.app.vectorstore import get_vector_db

[nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/ryanrodriguez/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /Users/ryanrodriguez/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!


In [17]:
system_role_prompt = """
    You are a helpful assistant that generates questions based on a given context.
"""

user_role_prompt = """
    Based on the following context about {query}, generate 5 relevant and specific questions.
    Make sure the questions can be answered using only the provided context.

    Context: {context}

    Generate 5 questions that test understanding of the material in the context.
    
    Return only a json object with the following format:
    {{
        "questions": ["question1", "question2", "question3", "question4", "question5"]
    }}
"""


In [18]:
chat_prompt = ChatPromptTemplate.from_messages([
    ("system", system_role_prompt),
    ("user", user_role_prompt)
])

openai_chat_model = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.7)

retriever = get_vector_db().as_retriever(search_kwargs={"k": 2})


In [19]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

simple_rag  = (
    {"context": retriever, "query": RunnablePassthrough(), "num_questions": RunnablePassthrough()}
    | chat_prompt
    | openai_chat_model
    | StrOutputParser()
)

In [22]:
raw_result = simple_rag.invoke("RAG")

In [23]:
import json
result = json.loads(raw_result)
result["questions"]

['What are the two main components of a typical RAG application?',
 'What is the purpose of the indexing component in a RAG application?',
 "What are the steps involved in the 'Load' phase of indexing?",
 'Why is splitting text into smaller chunks important in the context of RAG applications?',
 'How does the retrieval and generation component of a RAG application process user queries?']